In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [40]:
df = pd.read_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\eng_news_nocontent.csv')
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
0,"{'id': None, 'name': 'Trendmicro.com'}",Trend Micro,REDBALDKNIGHT/BRONZE BUTLER’s Daserf Backdoor ...,"REDBALDKNIGHT, also known as BRONZE BUTLER and...",https://blog.trendmicro.com/trendlabs-security...,https://blog.trendmicro.com/trendlabs-security...,2017-11-07T12:34:44Z,by Joey Chen and MingYen Hsieh (Threat Analyst...,Trendmicro.com
1,"{'id': None, 'name': 'Computerworld'}","Lucas Mearian, Lucas Mearian",What is blockchain? The most disruptive tech i...,Blockchain is poised to change IT in much the ...,https://www.computerworld.com/article/3191077/...,https://images.techhive.com/images/article/201...,2017-11-08T01:06:00Z,Blockchain is poised to change IT in much the ...,Computerworld
2,"{'id': None, 'name': 'hackernoon.com'}",Yaoqi Jia,Demystifying Hashgraph: Benefits and Challenges,"Recently, on our Zilliqa social channels a num...",https://hackernoon.com/demystifying-hashgraph-...,NaN,2017-11-08T17:21:35Z,"Recently, on our Zilliqa social channels a num...",hackernoon.com
3,"{'id': None, 'name': 'Topix.com'}",NaN,BitTorrent inventor announces eco-friendly Bit...,A Bitcoin transaction wastes as much electrici...,http://www.topix.com/tech/p2p/2017/11/bittorre...,http://picproxy.topixcdn.com/pics/up-,2017-11-08T23:21:17Z,A Bitcoin transaction wastes as much electrici...,Topix.com
4,"{'id': None, 'name': 'Ml-implode.com'}",NaN,Bitcoin Fork Called Off: Prices Soar After Seg...,The price of a bitcoin went on a tear on Wedne...,http://ml-implode.com/staticnews/2017-11-09_Bi...,NaN,2017-11-09T00:00:00Z,The price of a bitcoin went on a tear on Wedne...,Ml-implode.com
...,...,...,...,...,...,...,...,...,...
744427,"{'id': None, 'name': 'Biztoc.com'}",u.today,"Negatively Correlates With Bitcoin (BTC), What...",Shiba Inu continues to focus on utility as its...,https://biztoc.com/x/8248d6342c37d743,https://c.biztoc.com/p/8248d6342c37d743/s.webp,2023-09-30T21:50:10Z,Shiba Inu continues to focus on utility as its...,Biztoc.com
744428,"{'id': None, 'name': 'Slashdot.org'}",feedfeeder,Industry leaders and policymakers weigh in on ...,Industry leaders and policymakers weigh in on ...,https://slashdot.org/firehose.pl?op=view&amp;i...,NaN,2023-09-30T21:53:20Z,Sign up for the Slashdot newsletter! OR check ...,Slashdot.org
744429,"{'id': None, 'name': 'Investing.com'}",U.Today,Shiba Inu (SHIB) Negatively Correlates With Bi...,Shiba Inu (SHIB) Negatively Correlates With Bi...,https://www.investing.com/news/cryptocurrency-...,https://i-invdn-com.investing.com/news/LYNXNPE...,2023-09-30T22:00:22Z,U.Today - Dog-themed cryptocurrency is seeing ...,Investing.com
744430,"{'id': None, 'name': 'Ambcrypto.com'}",Benjamin Njiri,Can NEAR stay above $1?,"NEAR could be ripe for a price reversal, but t...",https://ambcrypto.com/?p=347645,https://statics.ambcrypto.com/wp-content/uploa...,2023-09-30T22:30:04Z,Disclaimer: The information presented does not...,Ambcrypto.com


In [41]:
df = df[(df['press'] == 'IT World Canada') | (df['press'] == 'Sap.com') | (df['press'] == 'iTnews')]
df

,source,author,title,description,url,urlToImage,publishedAt,content,press
1180,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,How to stop fraud,During the summer the accounting department at...,https://www.itworldcanada.com/article/how-to-s...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-14T12:00:19Z,During the summer the accounting department at...,IT World Canada
1212,"{'id': None, 'name': 'Sap.com'}",cormac reynolds,Choosing a VPN for Security: What You Need To ...,The internet has become more of a basic need f...,https://blogs.sap.com/2017/11/14/choosing-a-vp...,NaN,2017-11-14T13:17:23Z,The internet has become more of a basic need f...,Sap.com
1311,"{'id': None, 'name': 'Sap.com'}",Adam Winfield,How Blockchain’s ‘Web of Value’ Could Oust the...,"With Facebook, Google, Netflix, Spotify, and p...",https://blogs.sap.com/2017/11/14/how-blockchai...,NaN,2017-11-14T18:01:46Z,"With Facebook, Google, Netflix, Spotify, and p...",Sap.com
1527,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,Canada’s new cyber security strategy will be b...,A senior Canadian government official has give...,https://www.itworldcanada.com/article/398785-2...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-15T12:00:32Z,A senior Canadian government official has give...,IT World Canada
2455,"{'id': None, 'name': 'Sap.com'}",Paul Taylor,Blockchain: Digital Transformation’s ‘Cinderel...,Among the technologies and processes most clos...,https://blogs.sap.com/2017/11/17/blockchain-di...,NaN,2017-11-17T16:42:37Z,Among the technologies and processes most clos...,Sap.com
...,...,...,...,...,...,...,...,...,...
738114,"{'id': None, 'name': 'Sap.com'}",Peter Prus,Comment on STAY INFORMED: SAP ONE Support Laun...,After falling victim to a cryptocurrency inves...,https://blogs.sap.com/2023/09/15/stay-informed...,NaN,2023-09-17T15:21:42Z,"On May 25th, 2019 any Partner COE (VAR-D partn...",Sap.com
738165,"{'id': None, 'name': 'iTnews'}",NaN,Blockchain analysts suspect Lazarus Group hack...,From crypto exchange CoinEx.,https://www.itnews.com.au/news/blockchain-anal...,https://i.nextmedia.com.au/News/bitcoin_scam_c...,2023-09-17T20:52:00Z,Blockchain researchers say North Korea-linked ...,iTnews
742406,"{'id': None, 'name': 'Sap.com'}",Nagesh Polu,Empowering HR with AI: Revolutionizing Talent ...,From HR to HRAI In the fast-paced world of mod...,https://blogs.sap.com/2023/09/26/empowering-hr...,NaN,2023-09-26T20:56:45Z,From HR to HRAI\r\nIn the fast-paced world of ...,Sap.com
742425,"{'id': None, 'name': 'Sap.com'}",Berlinda Mensah,Comment on Empowering HR with AI: Revolutioniz...,From HR to HRAI In the fast-paced world of mod...,https://blogs.sap.com/2023/09/26/empowering-hr...,NaN,2023-09-26T21:45:08Z,From HR to HRAI\r\nIn the fast-paced world of ...,Sap.com


In [42]:
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import traceback

def get_contents(df):
    contents = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        url = row['url']
        press = row['press']
        
        # press 값에 따른 selector 설정
        if press == 'IT World Canada':
            selector = '.tagdiv-type .tdb-block-inner'
        elif press == 'Sap.com':
            selector = '.ds-editor-content'
        elif press == 'iTnews':
            selector = '#article-body'

        else:
            print(f"Unknown press value: {press} for URL: {url}")
            contents.append('')
            continue

        try:
            response = requests.get(url, headers=headers, timeout=60)  # 1분 timeout 설정
            soup = BeautifulSoup(response.text, "html.parser")

            article_elements = soup.select(selector)

            if article_elements:
                content = article_elements[0].get_text()
            else:
                content = ''

            contents.append(content)
        except (requests.Timeout, Exception) as e:
            if isinstance(e, requests.Timeout):
                print(f"Request timed out for URL: {url}")
            else:
                print(f"Error occurred: {e}")
                traceback.print_exc()
            contents.append('')

    return contents


In [26]:
# import requests
# from bs4 import BeautifulSoup
# import re
# from tqdm import tqdm

# def get_contents(yahoo):
#     contents = []

#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

#     for url in tqdm(yahoo['url']):
#         try:
#             response = requests.get(url, headers=headers)
#             soup = BeautifulSoup(response.text, "html.parser")

#             # 모든 클래스 이름 중에서 ".col-*-offset-1" 패턴을 가진 것들을 찾는다
#             classes = set()
#             for tag in soup.find_all(True, {'class': True}):
#                 for class_name in tag['class']:
#                     if re.match(r'col-\w+-offset-1', class_name):
#                         classes.add(class_name)

#             # 해당 패턴을 가진 클래스들로부터 내용을 추출한다
#             content = []
#             for class_name in classes:
#                 content.extend(soup.select(f".{class_name}"))
            
#             cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content))
#             contents.append(cleaned_content)
#         except:
#             contents.append('')

#     return contents


In [43]:
head = df.head()
head['content_full'] = get_contents(head)
head

100%|██████████| 5/5 [00:08<00:00,  1.78s/it]
<ipython-input-43-6286cb6d68db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  head['content_full'] = get_contents(head)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1180,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,How to stop fraud,During the summer the accounting department at...,https://www.itworldcanada.com/article/how-to-s...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-14T12:00:19Z,During the summer the accounting department at...,IT World Canada,During the summer the accounting department at...
1212,"{'id': None, 'name': 'Sap.com'}",cormac reynolds,Choosing a VPN for Security: What You Need To ...,The internet has become more of a basic need f...,https://blogs.sap.com/2017/11/14/choosing-a-vp...,NaN,2017-11-14T13:17:23Z,The internet has become more of a basic need f...,Sap.com,
1311,"{'id': None, 'name': 'Sap.com'}",Adam Winfield,How Blockchain’s ‘Web of Value’ Could Oust the...,"With Facebook, Google, Netflix, Spotify, and p...",https://blogs.sap.com/2017/11/14/how-blockchai...,NaN,2017-11-14T18:01:46Z,"With Facebook, Google, Netflix, Spotify, and p...",Sap.com,"\nWith Facebook, Google, Netflix, Spotify, and..."
1527,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,Canada’s new cyber security strategy will be b...,A senior Canadian government official has give...,https://www.itworldcanada.com/article/398785-2...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-15T12:00:32Z,A senior Canadian government official has give...,IT World Canada,A senior Canadian government official has give...
2455,"{'id': None, 'name': 'Sap.com'}",Paul Taylor,Blockchain: Digital Transformation’s ‘Cinderel...,Among the technologies and processes most clos...,https://blogs.sap.com/2017/11/17/blockchain-di...,NaN,2017-11-17T16:42:37Z,Among the technologies and processes most clos...,Sap.com,\nAmong the technologies and processes most cl...


In [44]:
# Adding the content to the DataFrame
df['content_full'] = get_contents(df)
df

100%|██████████| 1478/1478 [45:15<00:00,  1.84s/it] 
<ipython-input-44-0b636d97f36d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content_full'] = get_contents(df)


,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1180,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,How to stop fraud,During the summer the accounting department at...,https://www.itworldcanada.com/article/how-to-s...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-14T12:00:19Z,During the summer the accounting department at...,IT World Canada,During the summer the accounting department at...
1212,"{'id': None, 'name': 'Sap.com'}",cormac reynolds,Choosing a VPN for Security: What You Need To ...,The internet has become more of a basic need f...,https://blogs.sap.com/2017/11/14/choosing-a-vp...,NaN,2017-11-14T13:17:23Z,The internet has become more of a basic need f...,Sap.com,
1311,"{'id': None, 'name': 'Sap.com'}",Adam Winfield,How Blockchain’s ‘Web of Value’ Could Oust the...,"With Facebook, Google, Netflix, Spotify, and p...",https://blogs.sap.com/2017/11/14/how-blockchai...,NaN,2017-11-14T18:01:46Z,"With Facebook, Google, Netflix, Spotify, and p...",Sap.com,"\nWith Facebook, Google, Netflix, Spotify, and..."
1527,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,Canada’s new cyber security strategy will be b...,A senior Canadian government official has give...,https://www.itworldcanada.com/article/398785-2...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-15T12:00:32Z,A senior Canadian government official has give...,IT World Canada,A senior Canadian government official has give...
2455,"{'id': None, 'name': 'Sap.com'}",Paul Taylor,Blockchain: Digital Transformation’s ‘Cinderel...,Among the technologies and processes most clos...,https://blogs.sap.com/2017/11/17/blockchain-di...,NaN,2017-11-17T16:42:37Z,Among the technologies and processes most clos...,Sap.com,\nAmong the technologies and processes most cl...
...,...,...,...,...,...,...,...,...,...,...
738114,"{'id': None, 'name': 'Sap.com'}",Peter Prus,Comment on STAY INFORMED: SAP ONE Support Laun...,After falling victim to a cryptocurrency inves...,https://blogs.sap.com/2023/09/15/stay-informed...,NaN,2023-09-17T15:21:42Z,"On May 25th, 2019 any Partner COE (VAR-D partn...",Sap.com,"\nOn May 25th, 2019 any Partner COE (VAR-D par..."
738165,"{'id': None, 'name': 'iTnews'}",NaN,Blockchain analysts suspect Lazarus Group hack...,From crypto exchange CoinEx.,https://www.itnews.com.au/news/blockchain-anal...,https://i.nextmedia.com.au/News/bitcoin_scam_c...,2023-09-17T20:52:00Z,Blockchain researchers say North Korea-linked ...,iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
742406,"{'id': None, 'name': 'Sap.com'}",Nagesh Polu,Empowering HR with AI: Revolutionizing Talent ...,From HR to HRAI In the fast-paced world of mod...,https://blogs.sap.com/2023/09/26/empowering-hr...,NaN,2023-09-26T20:56:45Z,From HR to HRAI\r\nIn the fast-paced world of ...,Sap.com,\nFrom HR to HRAI\nIn the fast-paced world of ...
742425,"{'id': None, 'name': 'Sap.com'}",Berlinda Mensah,Comment on Empowering HR with AI: Revolutioniz...,From HR to HRAI In the fast-paced world of mod...,https://blogs.sap.com/2023/09/26/empowering-hr...,NaN,2023-09-26T21:45:08Z,From HR to HRAI\r\nIn the fast-paced world of ...,Sap.com,\nFrom HR to HRAI\nIn the fast-paced world of ...


In [285]:
head['url'].iloc[3]

'http://www.thestar.com.my/business/business-news/2017/11/13/bitcoin-tumble-erases-us$38b-as-rival-cryptocurrency-gains/'

In [296]:
head['content_full'].iloc[0]

"\n\n\n\n\n\nLipin Pappachan, Tech Times\n                    \n                21 November 2017, 09:11 am\n\n\n\nNearly $31 million worth of bitcoin currency was wiped off from cryptocurrency\xa0startup Tethers' Treasury account. The startup is in business with bitcoin exchanges and the hacker was able to move the bitcoin to an unauthorized account.\nTether has released the details of the attack and precautionary measures on its website for its users.\nDetails Of The Attack That Stole $31 Million\nOn Nov. 19, 2017, the Treasury account of Tether was compromised by a hacker. The bitcoins were maliciously transferred from Tether to the following address: 16tg2RJuEPtZooy18Wxn2me2RhUdC94N7r by the hacker. Although the fund address has now been flagged, the curtailment of further damage will depend on how the startup is able to control the spread of this fund in the broader ecosystem. Tether has requested people who trade in bitcoins to not to purchase USDT tokens from the above address.\n

In [45]:
empty_rows = df[df['content_full'] == '']
print(len(empty_rows))

125


In [46]:
empty_rows['url'].iloc[100]

'https://blogs.sap.com/2021/08/31/how-blockchain-and-machine-learning-are-altering-the-mobile-privacy-landscape/'

<!-- # empty_rows['url'].iloc[0] -->

In [47]:
ITWorldCanada = df[df['press'] == 'IT World Canada']
ITWorldCanada

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1180,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,How to stop fraud,During the summer the accounting department at...,https://www.itworldcanada.com/article/how-to-s...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-14T12:00:19Z,During the summer the accounting department at...,IT World Canada,During the summer the accounting department at...
1527,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,Canada’s new cyber security strategy will be b...,A senior Canadian government official has give...,https://www.itworldcanada.com/article/398785-2...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-15T12:00:32Z,A senior Canadian government official has give...,IT World Canada,A senior Canadian government official has give...
3348,"{'id': None, 'name': 'IT World Canada'}",Mandy Kovacs,Hashtag Trending – Uber’s fleet of driverless ...,"Uber is forming a fleet of driverless cars, Bi...",https://www.itworldcanada.com/article/hashtag-...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-21T12:33:05Z,"Uber is forming a fleet of driverless cars, Bi...",IT World Canada,"Uber is forming a fleet of driverless cars, Bi..."
5658,"{'id': None, 'name': 'IT World Canada'}",Mandy Kovacs,"Hashtag Trending – Cyber Monday sets records, ...","Cyber Monday 2017 could set new records, Jeff ...",https://www.itworldcanada.com/article/hashtag-...,https://www.itworldcanada.com/wp-content/uploa...,2017-11-28T12:23:50Z,"Cyber Monday 2017 could set new records, Jeff ...",IT World Canada,"Cyber Monday 2017 could set new records, Jeff ..."
9036,"{'id': None, 'name': 'IT World Canada'}",Brian Jackson,"Hashtag Trending – Alexa in Canada, unlocked s...",Amazon Echo ships to Canada. Free unlocked sma...,https://www.itworldcanada.com/article/hashtag-...,https://www.itworldcanada.com/wp-content/theme...,2017-12-04T13:05:20Z,Amazon Echo ships to Canada. Free unlocked sma...,IT World Canada,Amazon Echo ships to Canada. Free unlocked sma...
...,...,...,...,...,...,...,...,...,...,...
731142,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,"Cyber Security Today, Sept. 1, 2023 – Celebrat...",Celebrate Women in Cybersecurity. Welcome to C...,https://www.itworldcanada.com/article/cyber-se...,https://i.itworldcanada.com/wp-content/uploads...,2023-09-01T11:47:38Z,Celebrate Women in Cybersecurity.\r\nWelcome t...,IT World Canada,Celebrate Women in Cybersecurity.\nWelcome to ...
732564,"{'id': None, 'name': 'IT World Canada'}",Paul Barker,Need for ‘digitally savvy’ corporate boards mo...,No group has more influence on how an organiza...,https://www.itworldcanada.com/article/need-for...,https://i.itworldcanada.com/wp-content/uploads...,2023-09-05T15:10:55Z,No group has more influence on how an organiza...,IT World Canada,No group has more influence on how an organiza...
734134,"{'id': None, 'name': 'IT World Canada'}",Jim Love,Hashtag Trending Sep.8- ChatGPT traffic declin...,ChatGPT traffic declines for the third month i...,https://www.itworldcanada.com/article/hashtag-...,https://i.itworldcanada.com/wp-content/uploads...,2023-09-08T09:00:53Z,ChatGPT traffic declines for the third month i...,IT World Canada,ChatGPT traffic declines for the third month i...
735126,"{'id': None, 'name': 'IT World Canada'}",Howard Solomon,"Cyber Security Today, Sept. 11, 2023 – Warning...","Warnings from Cisco, a huge DDoS attack and mo...",https://www.itworldcanada.com/article/cyber-se...,https://i.itworldcanada.com/wp-content/uploads...,2023-09-11T12:03:52Z,"Warnings from Cisco, a huge DDoS attack and mo...",IT World Canada,"Warnings from Cisco, a huge DDoS attack and mo..."


In [48]:
Sap = df[df['press'] == 'Sap.com']
Sap

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
1212,"{'id': None, 'name': 'Sap.com'}",cormac reynolds,Choosing a VPN for Security: What You Need To ...,The internet has become more of a basic need f...,https://blogs.sap.com/2017/11/14/choosing-a-vp...,NaN,2017-11-14T13:17:23Z,The internet has become more of a basic need f...,Sap.com,
1311,"{'id': None, 'name': 'Sap.com'}",Adam Winfield,How Blockchain’s ‘Web of Value’ Could Oust the...,"With Facebook, Google, Netflix, Spotify, and p...",https://blogs.sap.com/2017/11/14/how-blockchai...,NaN,2017-11-14T18:01:46Z,"With Facebook, Google, Netflix, Spotify, and p...",Sap.com,"\nWith Facebook, Google, Netflix, Spotify, and..."
2455,"{'id': None, 'name': 'Sap.com'}",Paul Taylor,Blockchain: Digital Transformation’s ‘Cinderel...,Among the technologies and processes most clos...,https://blogs.sap.com/2017/11/17/blockchain-di...,NaN,2017-11-17T16:42:37Z,Among the technologies and processes most clos...,Sap.com,\nAmong the technologies and processes most cl...
3020,"{'id': None, 'name': 'Sap.com'}",James Zdralek,"Dead Accounts, Dead Money and Hoarding (5th in...",What if the world ran on new classes of digita...,https://blogs.sap.com/2017/11/20/dead-accounts...,NaN,2017-11-20T16:20:40Z,What if the world ran on new classes of digita...,Sap.com,\nWhat if the world ran on new classes of digi...
3024,"{'id': None, 'name': 'Sap.com'}",Paul Taylor,Blockchain: Digital Transformation’s Cinderell...,Blockchain technology is not widely understood...,https://news.sap.com/blockchain-digital-transf...,https://news.sap.com/wp-content/blogs.dir/1/fi...,2017-11-20T16:30:21Z,Among the technologies and processes most clos...,Sap.com,
...,...,...,...,...,...,...,...,...,...,...
738113,"{'id': None, 'name': 'Sap.com'}",Peter Prus,Comment on Preparing for PCOE: SAP Commerce Su...,After falling victim to a cryptocurrency inves...,https://blogs.sap.com/2023/09/15/preparing-for...,NaN,2023-09-17T15:03:36Z,SAP Sell Partners interested in becoming suppo...,Sap.com,
738114,"{'id': None, 'name': 'Sap.com'}",Peter Prus,Comment on STAY INFORMED: SAP ONE Support Laun...,After falling victim to a cryptocurrency inves...,https://blogs.sap.com/2023/09/15/stay-informed...,NaN,2023-09-17T15:21:42Z,"On May 25th, 2019 any Partner COE (VAR-D partn...",Sap.com,"\nOn May 25th, 2019 any Partner COE (VAR-D par..."
742406,"{'id': None, 'name': 'Sap.com'}",Nagesh Polu,Empowering HR with AI: Revolutionizing Talent ...,From HR to HRAI In the fast-paced world of mod...,https://blogs.sap.com/2023/09/26/empowering-hr...,NaN,2023-09-26T20:56:45Z,From HR to HRAI\r\nIn the fast-paced world of ...,Sap.com,\nFrom HR to HRAI\nIn the fast-paced world of ...
742425,"{'id': None, 'name': 'Sap.com'}",Berlinda Mensah,Comment on Empowering HR with AI: Revolutioniz...,From HR to HRAI In the fast-paced world of mod...,https://blogs.sap.com/2023/09/26/empowering-hr...,NaN,2023-09-26T21:45:08Z,From HR to HRAI\r\nIn the fast-paced world of ...,Sap.com,\nFrom HR to HRAI\nIn the fast-paced world of ...


In [49]:
iTnews = df[df['press'] == 'iTnews']
iTnews

,source,author,title,description,url,urlToImage,publishedAt,content,press,content_full
2769,"{'id': None, 'name': 'iTnews'}",NaN,Fork talk lifts Bitcoin to all-time high,"Near $11,000.",https://www.itnews.com.au/news/fork-talk-lifts...,http://i.nextmedia.com.au/News/20170306092432_...,2017-11-19T23:25:00Z,Bitcoin hit an all-time high just below US$800...,iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
6769,"{'id': None, 'name': 'iTnews'}",NaN,"Bitcoin tops US$11,000 after massive surge",Is the bubble about to burst?,https://www.itnews.com.au/news/bitcoin-tops-us...,https://i.nextmedia.com.au/News/2017-11-29T034...,2017-11-29T20:55:00Z,"Bitcoin has zoomed past US$11,000 (A$14,519) t...",iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
7521,"{'id': None, 'name': 'iTnews'}",NaN,Bitcoin loses over a fifth of its value in les...,Comes down from soaring heights.,https://www.itnews.com.au/news/bitcoin-loses-o...,https://i.nextmedia.com.au/News/2017-11-28T002...,2017-11-30T20:46:00Z,"Bitcoin slid to as low as US$9000 (A$11,899) i...",iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
10692,"{'id': None, 'name': 'iTnews'}",NaN,"Bitcoin surges above US$12,000 to record high",On relentless demand.,https://www.itnews.com.au/news/bitcoin-surges-...,https://i.nextmedia.com.au/News/2017-11-30T140...,2017-12-06T20:43:00Z,Bitcoin extended its eye-popping rally on Wedn...,iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
10788,"{'id': None, 'name': 'iTnews'}",NaN,The ASX is going ahead with a blockchain-based...,CHESS overhaul will use distributed ledger tec...,https://www.itnews.com.au/news/the-asx-is-goin...,https://i.nextmedia.com.au/News/20160425025258...,2017-12-06T22:58:00Z,The Australian Securities Exchange has made th...,iTnews,\nThe Australian Securities Exchange has made ...
...,...,...,...,...,...,...,...,...,...,...
716273,"{'id': None, 'name': 'iTnews'}",NaN,Cloud company assisted 17 different government...,US researchers publish findings.,https://www.itnews.com.au/news/cloud-company-a...,https://i.nextmedia.com.au/News/2023-08-01T100...,2023-08-01T19:37:00Z,An obscure cloud service company has been prov...,iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
720230,"{'id': None, 'name': 'iTnews'}",NaN,US Fed clarifies process for banks to transact...,Should obtain permission in writing.,https://www.itnews.com.au/news/us-fed-clarifie...,https://i.nextmedia.com.au/News/stablecoin.jpg,2023-08-09T20:45:00Z,State banks that are a member of the US Federa...,iTnews,\n\n\n\n\n\n\n\r\n\r\n ...
728403,"{'id': None, 'name': 'iTnews'}",NaN,SIM swap against Kroll staffer leads to data b...,Affecting claimants of crypto companies subjec...,https://www.itnews.com.au/news/sim-swap-agains...,https://i.nextmedia.com.au/News/nextpacific_cr...,2023-08-28T02:40:00Z,"Kroll, a claims administration company keeping...",iTnews,"\nKroll, a claims administration company keepi..."
732728,"{'id': None, 'name': 'iTnews'}",NaN,"Senate committee urges digital assets bill ""no...","Coalition sees crypto regulation bound for ""th...",https://www.itnews.com.au/news/senate-committe...,https://i.nextmedia.com.au/News/CRN-690-Austra...,2023-09-05T20:46:00Z,A senate committee has suggested the federal g...,iTnews,\nA senate committee has suggested the federal...


In [52]:
iTnews.to_csv(r'C:\Users\user\boyu571_github\01_Kakaobank_SKKU_Research_23\foreign_news_data\iTnews_199.csv', encoding='utf-8-sig')

In [ ]:
#article-13715 > div.post__content-wrapper > div.post-content